#Expected Parrot, personality developer and projects 5 factors of a firm from a rating of 1-10 given a list of characterisitics.

* will give model per CEO, per year: percent of gender, race, age, emotion (happy, sad, surprise, neutral) *

-> a way to turn rankings (1-10) into actual firm stock return projections during that year?

-> then compare actual performance with projections.



In [ ]:
! uv pip install edsl feedparser -q # uv is a fast Python package installer (written in Rust)

In [ ]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import requests
import feedparser
import math
import time
import random
import re
import yfinance as yf
from typing import Iterable, Union, Tuple, Dict, Any, List
from tqdm import tqdm
import yfinance as yf
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from edsl import QuestionFreeText, ScenarioList, Scenario, Model, Agent
from edsl import AgentList, Results
from google.colab import userdata
from google.colab import drive
from io import StringIO

os.environ['EXPECTED_PARROT_API_KEY'] = userdata.get('ExpectedParrot')

# File in/out access
drive.mount('/content/drive')
dir = "/content/drive/MyDrive/Colab Notebooks/Axion FIN 377 Final Project/Expected Parrot/" # Ensure dir is available

# EDSL model

m = Model("gemini-2.0-flash", parameters = {'max_tokens':10, 'temperature':1})
m

Mounted at /content/drive


,key,value
0,model,gemini-2.0-flash
1,parameters:max_tokens,10
2,parameters:temperature,1
3,inference_service,google


## Inputs

In [ ]:
# Where to put the files
save_dir = file_path = f'/content/drive/MyDrive/Colab Notebooks/Axion FIN 377 Final Project/ExpectedParrot/'
os.makedirs(save_dir, exist_ok=True)

# firms and headline parameters
TEST_LIMIT_CEOS = 409       # Set to 20-50 while developing code and you'll only use the first TEST_LIMIT_FIRMS in the sp500 list. Set to None to use all firms in the SP500.

# survey parameters
num_agents = 20              # survey size
prior_result_uuid = "8d6b0de3-ab28-4388-bda7-eccad0e88b28"
                             # You can reload prior survey results with a numeric ID from the
                             # go to https://www.expectedparrot.com/home/remote-inference
                             # find the job you want, click on the "View" button to see the progress report,
                             # then click Pull, then copy the string from inside the Pull code, e.g.:
                             # You see: results = Results.pull("b6bc0835-0fd8-488b-b560-1ab686a59ace")
                             # Put prior_result_uuid = "b6bc0835-0fd8-488b-b560-1ab686a59ace"



In [ ]:
# we will run N surveys:
TEST_LIMIT_CEOS*num_agents

8180

## Get a list of firms

* I think this is where the csv file from cecy would come in to play *

In [ ]:
#Load data from CEO deepface csv
dir = "/content/drive/MyDrive/Colab Notebooks/Axion FIN 377 Final Project/ExpectedParrot/"

df = pd.read_csv(dir+"ceo_face_analysis.csv",
                 names= ['Ticker', 'CEO', 'Year', 'Age', 'dominant_gender', 'dominant_race', 'dominant_emotion',
                         'Woman', 'Man', 'asian', 'indian', 'black', 'white', 'middle eastern', 'latino hispanic', 'angry',
                         'disgust', 'fear', 'happy', 'sad', 'surprise','neutral'])

In [ ]:
ceo_df = pd.read_csv(dir+"ceo_face_analysis.csv") # Added this line to define firm_df

In [ ]:
ceo_df

,Ticker,CEO,Year,Age,dominant_gender,dominant_race,dominant_emotion,Woman,Man,asian,...,white,middle eastern,latino hispanic,angry,disgust,fear,happy,sad,surprise,neutral
0,AAPL,Tim Cook,2011,28,Man,white,happy,0.003152,99.996860,2.648976e-01,...,77.172080,8.134223,14.040607,6.092998e-09,1.726371e-14,5.901342e-11,99.608030,1.010190e-06,5.153031e-06,0.391964
1,AAPL,Tim Cook,2012,42,Man,white,happy,0.014313,99.985680,5.961301e-04,...,98.712980,0.831979,0.451916,9.649724e-05,2.776464e-14,2.548410e-10,99.999560,3.069039e-08,3.228852e-12,0.000347
2,AAPL,Tim Cook,2013,46,Man,white,sad,0.005277,99.994720,3.110619e-04,...,98.948395,0.634838,0.415289,1.739757e+01,1.566626e-06,1.984285e-01,0.087631,4.616572e+01,1.444513e-02,36.136204
3,AAPL,Tim Cook,2014,43,Man,white,angry,0.010568,99.989426,1.752497e+00,...,46.372734,33.576042,12.853511,7.251926e+01,9.066945e-09,8.379727e-03,0.040729,8.794817e+00,1.569221e-05,18.636806
4,AAPL,Tim Cook,2015,37,Man,white,happy,0.003619,99.996376,2.234547e+00,...,55.381410,19.049858,19.705553,1.382578e-05,3.385820e-07,3.494481e-02,99.914200,2.773857e-04,5.245459e-04,0.050039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,ENPH,Paul Nahi,2016,27,Man,middle eastern,happy,0.004364,99.995636,1.364800e-01,...,14.005911,63.888653,17.773792,7.707024e-08,2.222067e-12,1.269394e-08,98.081170,3.692474e-05,2.240229e-05,1.918766
406,ENPH,Paul Nahi,2017,27,Man,middle eastern,happy,0.004364,99.995636,1.364800e-01,...,14.005911,63.888653,17.773792,7.707024e-08,2.222067e-12,1.269394e-08,98.081170,3.692474e-05,2.240229e-05,1.918766
407,TER,Michael A. Bradley,2012,46,Man,white,happy,0.016481,99.983510,1.296493e-08,...,99.995990,0.002925,0.001088,7.365736e-05,8.984494e-17,3.659834e-09,99.999910,1.074656e-08,1.735846e-11,0.000028
408,TYL,John S. Marr Jr.,2018,49,Man,white,happy,0.053745,99.946250,3.494762e-02,...,88.408676,6.072889,5.442528,5.351635e-10,4.552022e-17,1.214795e-11,99.763580,1.496498e-06,4.985265e-07,0.236423


## Make a Panel of Fake people

In [ ]:
sex = ["male", "female"]
race = ["white", "black", "hispanic", "asian", "indian", "middle eastern"]
age = ["18-33","34-50","50-65","65+" ]

income = [
    "under $50,000",
    "$50,000 to $99,999",
    "$100,000 to 200,000",
    "200,000 or more"
]
political_ideology = ["liberal", "independent", "conservative"]

# how many possible combinations?
print(len(sex) * len(race) * len(age) * len(income) * len(political_ideology))

576


In [ ]:
def generate_random_agents(num_agents):
    '''
    Each simulated agent will have random traits. Notes:
    1. random.choice() chooses from a list with equal probability.
       You can switch to random.choices(race, weights = [50,15,35])
       to specify more realistic draws. random.choices, not random.choice
    2. I did not set a "seed" here - every time we run this, it gets a
       new result. This is fine for experimenting, but bad for replication.
       Ask Gemini to set a seed so you always get the same result.
    '''
    random.seed(42) # Set a seed for reproducibility
    agents = []
    for _ in range(num_agents):
        agent_traits = {
            "sex": random.choices(sex),
            "race": random.choices(race),
            "age": random.choices(age),
            "income": random.choices(income),
            "political_ideology": random.choices(political_ideology),
        }
        agents.append(Agent(traits=agent_traits,
                            traits_presentation_template = " I am a {{ age }} year-old {{ race }} {{ sex }}. I make {{ income }} per year and am politically {{ political_ideology }}."
        ))

    return AgentList(agents)

In [ ]:
agents = generate_random_agents(num_agents)

In [ ]:
# ABCD! always be checking your data
# Show an agent's prompt to check
a = agents[0]
a.agent_persona.render(primary_replacement = a.traits).text

" I am a ['34-50'] year-old ['white'] ['female']. I make ['under $50,000'] per year and am politically ['conservative']."

## Design the Survey

In [ ]:
from edsl import QuestionNumerical, QuestionFreeText, ScenarioList, Scenario, Survey

q1 = QuestionNumerical(
    question_name = 'Sentiment',
    question_text = '''Take on that person's likely worldview, communication style, and financial expertise.

As this person,  how would you interpret the effect of the following financial news on the company's stock price?

HEADLINE: {{ headline }}
COMPANY: {{ company }}
DATE: {{ date }}

Provide a sentiment score from 1 (very negative) to 10 (very positive) about the company's future stock price based on your interpretation of the article.'''
)

survey = Survey(questions = [q1])

scenarios = ScenarioList([
    Scenario({"headline":  row['Headline'],
     "company": row['Ticker'],
     'date': row['Date']}) for index, row in all_headlines_df.iterrows()
]
)



In [ ]:
scenarios[10] # abcd

In [ ]:
job = q1.by(scenarios).by(agents).by(m)

In [ ]:
# job.show_prompts()  # slow and big. best done on a smaller sample of scenarios

# Run the survey

In [ ]:
# run the survey or load it
if not prior_result_uuid:
    results = job.run()
else:
    results = Results.pull(prior_result_uuid)

# we want this as a dataframe: results.to_pandas() should work, but

# The results object here is big enough to cause memory issues in Colab so
# 1. make it smaller by keep just the columns we need
results = results.select("headline", "company", "date", "Sentiment")

# 2. convert it to pands in chunks (if you do it all at once, it's requires
#     having the results object and the df version in memory at once... double the space)

def chunked_iterable(iterable, size):
    for i in range(0, len(iterable), size):
        yield iterable[i : i + size]

result_df = None

for batch in chunked_iterable(results, 10000):
    new_results = batch.select("headline", "company", "date", "Sentiment").to_pandas()
    if results is None:
        result_df = new_results
    else:
        result_df = pd.concat([result_df, new_results])

del results # free up the space

# Task
Generate a new EDSL survey to evaluate CEOs based on their demographic and emotion characteristics. This survey should include four new numerical questions (Projected risk-taking, Projected leadership strength, Projected communication skill, and Projected financial stewardship), each rated on a 1-5 scale. Create scenarios for the survey by iterating through the `ceo_df` DataFrame, extracting relevant CEO demographic and emotion characteristics, and limiting the number of CEOs to `TEST_LIMIT_CEOS`. After running this new survey, update the `results.select()` call in cell `sus7An1Cdyll` to include the new question names for result extraction.

## Define New Survey Questions and Scenarios

### Subtask:
Create four QuestionNumerical objects for Projected risk-taking, Projected leadership strength, Projected communication skill, and Projected financial stewardship, each with a 1-5 rating scale. Then, generate a ScenarioList by iterating through the ceo_df, extracting relevant CEO demographic and emotion characteristics for each entry, applying TEST_LIMIT_CEOS if specified. Finally, assemble these questions, scenarios, and agents into a new EDSL job object in the selected cell.


**Reasoning**:
First, I will define the four `QuestionNumerical` objects as specified, with their respective question texts and rating scales. Then, I will combine these questions into a new `Survey` object. Next, I will prepare the scenarios by iterating through the `ceo_df` dataframe, extracting the required CEO characteristics, and applying the `TEST_LIMIT_CEOS` limit if set. Finally, I will assemble these components into an EDSL job object. All these actions can be performed in a single code block.



In [ ]:
from edsl import QuestionNumerical, ScenarioList, Scenario, Survey

q_risk_taking = QuestionNumerical(
    question_name='Projected_risk_taking',
    question_text='Given the CEO: {{ CEO }} (Year: {{ Year }}, Age: {{ Age }}, Gender: {{ dominant_gender }}, Race: {{ dominant_race }}, Emotion: {{ dominant_emotion }}), rate their projected risk-taking on a scale of 1 (very low) to 5 (very high).',
    min_value=1,
    max_value=5
)

q_leadership_strength = QuestionNumerical(
    question_name='Projected_leadership_strength',
    question_text='Given the CEO: {{ CEO }} (Year: {{ Year }}, Age: {{ Age }}, Gender: {{ dominant_gender }}, Race: {{ dominant_race }}, Emotion: {{ dominant_emotion }}), rate their projected leadership strength on a scale of 1 (very weak) to 5 (very strong).',
    min_value=1,
    max_value=5
)

q_communication_skill = QuestionNumerical(
    question_name='Projected_communication_skill',
    question_text='Given the CEO: {{ CEO }} (Year: {{ Year }}, Age: {{ Age }}, Gender: {{ dominant_gender }}, Race: {{ dominant_race }}, Emotion: {{ dominant_emotion }}), rate their projected communication skill on a scale of 1 (very poor) to 5 (very excellent).',
    min_value=1,
    max_value=5
)

q_financial_stewardship = QuestionNumerical(
    question_name='Projected_financial_stewardship',
    question_text='Given the CEO: {{ CEO }} (Year: {{ Year }}, Age: {{ Age }}, Gender: {{ dominant_gender }}, Race: {{ dominant_race }}, Emotion: {{ dominant_emotion }}), rate their projected financial stewardship on a scale of 1 (very irresponsible) to 5 (very responsible).',
    min_value=1,
    max_value=5
)

ceo_survey = Survey(questions=[
    q_risk_taking,
    q_leadership_strength,
    q_communication_skill,
    q_financial_stewardship
])

# Apply TEST_LIMIT_CEOS if specified
if TEST_LIMIT_CEOS is not None:
    limited_ceo_df = ceo_df.head(TEST_LIMIT_CEOS)
else:
    limited_ceo_df = ceo_df

ceo_scenarios = ScenarioList([
    Scenario({
        "CEO": row['CEO'],
        "Year": row['Year'],
        "Age": row['Age'],
        "dominant_gender": row['dominant_gender'],
        "dominant_race": row['dominant_race'],
        "dominant_emotion": row['dominant_emotion']
    }) for index, row in limited_ceo_df.iterrows()
])

ceo_job = ceo_survey.by(ceo_scenarios).by(agents).by(m)

print(f"Created {len(ceo_survey.questions)} survey questions.")
print(f"Generated {len(ceo_scenarios)} scenarios from ceo_df.")
print("Created EDSL job object 'ceo_job'.")

Created 4 survey questions.
Generated 409 scenarios from ceo_df.
Created EDSL job object 'ceo_job'.


## Run new CEO survey and process results

### Subtask:
Run the `ceo_job` survey, then select the relevant columns (CEO, Year, Age, dominant_gender, dominant_race, dominant_emotion, and the four new question names) and convert the results to a pandas DataFrame, potentially using chunking for memory management.


**Reasoning**:
The subtask requires running the newly created EDSL job, selecting specific columns from its results, and converting these results into a pandas DataFrame using a chunking method to handle potential memory constraints, similar to the previous survey processing.



In [ ]:
if not prior_result_uuid:
    ceo_results = ceo_job.run()
else:
    # For simplicity, if prior_result_uuid is set, we'll assume it's for the old survey
    # and for the new CEO survey, we'll always run it for now. If a prior_result_uuid
    # was specifically for this CEO survey, it would need a different logic path.
    print("Running CEO survey as 'prior_result_uuid' is for a different survey.")
    ceo_results = ceo_job.run()

# Define the columns to select, including CEO characteristics and new question names
columns_to_select = [
    "CEO",
    "Year",
    "Age",
    "dominant_gender",
    "dominant_race",
    "dominant_emotion",
    "Projected_risk_taking",
    "Projected_leadership_strength",
    "Projected_communication_skill",
    "Projected_financial_stewardship"
]

# Select the necessary columns
ceo_results_selected = ceo_results.select(*columns_to_select)

ceo_result_df = None

# Convert to pandas DataFrame in chunks
for batch in chunked_iterable(ceo_results_selected, 10000):
    new_batch_df = batch.to_pandas()
    if ceo_result_df is None:
        ceo_result_df = new_batch_df
    else:
        ceo_result_df = pd.concat([ceo_result_df, new_batch_df])

del ceo_results # Free up memory
del ceo_results_selected # Free up memory

print(f"Successfully created ceo_result_df with {len(ceo_result_df)} rows and {len(ceo_result_df.columns)} columns.")
print(ceo_result_df.head())

Running CEO survey as 'prior_result_uuid' is for a different survey.


NameError: name 'chunked_iterable' is not defined

**Reasoning**:
The error `NameError: name 'chunked_iterable' is not defined` indicates that the `chunked_iterable` function was not available in the scope of the cell where it was called. This function was defined in a previous cell but likely got removed or was not executed before this cell. To fix this, I will include the definition of `chunked_iterable` directly within the current code block before its first use.



In [ ]:
def chunked_iterable(iterable, size):
    for i in range(0, len(iterable), size):
        yield iterable[i : i + size]


if not prior_result_uuid:
    ceo_results = ceo_job.run()
else:
    # For simplicity, if prior_result_uuid is set, we'll assume it's for the old survey
    # and for the new CEO survey, we'll always run it for now. If a prior_result_uuid
    # was specifically for this CEO survey, it would need a different logic path.
    print("Running CEO survey as 'prior_result_uuid' is for a different survey.")
    ceo_results = ceo_job.run()

# Define the columns to select, including CEO characteristics and new question names
columns_to_select = [
    "CEO",
    "Year",
    "Age",
    "dominant_gender",
    "dominant_race",
    "dominant_emotion",
    "Projected_risk_taking",
    "Projected_leadership_strength",
    "Projected_communication_skill",
    "Projected_financial_stewardship"
]

# Select the necessary columns
ceo_results_selected = ceo_results.select(*columns_to_select)

ceo_result_df = None

# Convert to pandas DataFrame in chunks
for batch in chunked_iterable(ceo_results_selected, 10000):
    new_batch_df = batch.to_pandas()
    if ceo_result_df is None:
        ceo_result_df = new_batch_df
    else:
        ceo_result_df = pd.concat([ceo_result_df, new_batch_df])

del ceo_results # Free up memory
del ceo_results_selected # Free up memory

print(f"Successfully created ceo_result_df with {len(ceo_result_df)} rows and {len(ceo_result_df.columns)} columns.")
print(ceo_result_df.head())

Running CEO survey as 'prior_result_uuid' is for a different survey.


Exception Type,Service,Model,Question,Count
QuestionAnswerValidationError,google,gemini-2.0-flash,Projected_financial_stewardship,2


KeyboardInterrupt: 

# Task
Confirm the successful creation and display of the `ceo_result_df` containing the processed survey results.

## Run CEO Survey and Process Results

### Subtask:
Re-execute the code that defines the `chunked_iterable` function and then runs the `ceo_job` to gather all survey responses. This will attempt to resume the survey if it was interrupted or start it anew if necessary. After the survey is complete, the results will be selected, processed in chunks, and stored in the `ceo_result_df` DataFrame.


**Reasoning**:
The previous attempt to run the CEO survey and process results failed due to a `NameError` (chunked_iterable not defined) and a `KeyboardInterrupt` during survey execution. To fix this, I will provide the complete code block that defines the `chunked_iterable` function and then proceeds to run the `ceo_job` and process its results in chunks, ensuring all necessary components are present and executed together.



In [ ]:
def chunked_iterable(iterable, size):
    for i in range(0, len(iterable), size):
        yield iterable[i : i + size]


if not prior_result_uuid:
    ceo_results = ceo_job.run()
else:
    # For simplicity, if prior_result_uuid is set, we'll assume it's for the old survey
    # and for the new CEO survey, we'll always run it for now. If a prior_result_uuid
    # was specifically for this CEO survey, it would need a different logic path.
    print("Running CEO survey as 'prior_result_uuid' is for a different survey.")
    ceo_results = ceo_job.run()

# Define the columns to select, including CEO characteristics and new question names
columns_to_select = [
    "CEO",
    "Year",
    "Age",
    "dominant_gender",
    "dominant_race",
    "dominant_emotion",
    "Projected_risk_taking",
    "Projected_leadership_strength",
    "Projected_communication_skill",
    "Projected_financial_stewardship"
]

# Select the necessary columns
ceo_results_selected = ceo_results.select(*columns_to_select)

ceo_result_df = None

# Convert to pandas DataFrame in chunks
for batch in chunked_iterable(ceo_results_selected, 10000):
    new_batch_df = batch.to_pandas()
    if ceo_result_df is None:
        ceo_result_df = new_batch_df
    else:
        ceo_result_df = pd.concat([ceo_result_df, new_batch_df])

del ceo_results # Free up memory
del ceo_results_selected # Free up memory

print(f"Successfully created ceo_result_df with {len(ceo_result_df)} rows and {len(ceo_result_df.columns)} columns.")
print(ceo_result_df.head())

Running CEO survey as 'prior_result_uuid' is for a different survey.


Successfully created ceo_result_df with 8180 rows and 10 columns.
  scenario.CEO  scenario.Year  scenario.Age scenario.dominant_gender  \
0     Tim Cook           2011            28                      Man   
1     Tim Cook           2012            42                      Man   
2     Tim Cook           2013            46                      Man   
3     Tim Cook           2014            43                      Man   
4     Tim Cook           2015            37                      Man   

  scenario.dominant_race scenario.dominant_emotion  \
0                  white                     happy   
1                  white                     happy   
2                  white                       sad   
3                  white                     angry   
4                  white                     happy   

   answer.Projected_risk_taking  answer.Projected_leadership_strength  \
0                             2                                   3.0   
1                             

In [1]:
ceo_result_df

NameError: name 'ceo_result_df' is not defined

In [ ]:
#show me how many unique scenario.CEO values there

In [ ]:
output_filename = os.path.join(dir, 'ceo_survey_results.csv')
ceo_result_df.to_csv(output_filename, index=False)
print(f"DataFrame saved to {output_filename}")

DataFrame saved to /content/drive/MyDrive/Colab Notebooks/Axion FIN 377 Final Project/ExpectedParrot/ceo_survey_results.csv
